信用付

In [21]:
# 接口地址
test_url = "https://test.lianlianpay-inc.com/mpay-openapi"
# 支付申请
pay_apply_api = "/v1/creditpay/stagecode/apply"
# 私钥
private_key= "MIICdwIBADANBgkqhkiG9w0BAQEFAASCAmEwggJdAgEAAoGBAOilN4tR7HpNYvSBra/DzebemoAiGtGeaxa+qebx/O2YAdUFPI+xTKTX2ETyqSzGfbxXpmSax7tXOdoa3uyaFnhKRGRvLdq1kTSTu7q5s6gTryxVH2m62Py8Pw0sKcuuV0CxtxkrxUzGQN+QSxf+TyNAv5rYi/ayvsDgWdB3cRqbAgMBAAECgYEAj02d/jqTcO6UQspSY484GLsL7luTq4Vqr5L4cyKiSvQ0RLQ6DsUG0g+Gz0muPb9ymf5fp17UIyjioN+ma5WquncHGm6ElIuRv2jYbGOnl9q2cMyNsAZCiSWfR++op+6UZbzpoNDiYzeKbNUz6L1fJjzCt52w/RbkDncJd2mVDRkCQQD/Uz3QnrWfCeWmBbsAZVoM57n01k7hyLWmDMYoKh8vnzKjrWScDkaQ6qGTbPVL3x0EBoxgb/smnT6/A5XyB9bvAkEA6UKhP1KLi/ImaLFUgLvEvmbUrpzY2I1+jgdsoj9Bm4a8K+KROsnNAIvRsKNgJPWd64uuQntUFPKkcyfBV1MXFQJBAJGs3Mf6xYVIEE75VgiTyx0x2VdoLvmDmqBzCVxBLCnvmuToOU8QlhJ4zFdhA1OWqOdzFQSw34rYjMRPN24wKuECQEqpYhVzpWkA9BxUjli6QUo0feT6HUqLV7O8WqBAIQ7X/IkLdzLa/vwqxM6GLLMHzylixz9OXGZsGAkn83GxDdUCQA9+pQOitY0WranUHeZFKWAHZszSjtbe6wDAdiKdXCfig0/rOdxAODCbQrQs7PYy1ed8DuVQlHPwRGtokVGHATU="
# 连连公钥
ll_public_key= "MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQCSS/DiwdCf/aZsxxcacDnooGph3d2JOj5GXWi+q3gznZauZjkNP8SKl3J2liP0O6rU/Y/29+IUe+GTMhMOFJuZm1htAtKiu5ekW0GlBMWxf4FPkYlQkPE0FtaoMP3gYfh+OwI+fIRrpW3ySn3mScnc6Z700nU/VYrRkfcSCbSnRwIDAQAB"
# 商户号
mch_id = '302112250000006404'

encoding = 'utf-8'


In [22]:
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import MD5

import base64
import requests
import json
from time import strftime, localtime

In [49]:
# 测试接口地址
# test_url = "http://localhost:8085/"

签名工具

In [19]:
# 数据签名

def rsa_sign(data):

    # 数据求md5
    hash_obj = MD5.new(data.encode(encoding)).hexdigest()

    
    # Md5withRsaSign
    pri_key = RSA.importKey(base64.b64decode(private_key))
    signer = PKCS1_v1_5.new(pri_key)
    
    md5_obj = MD5.new(hash_obj.encode(encoding))
    signature = base64.b64encode(signer.sign(md5_obj))
    return signature

sign_res = rsa_sign('xxdda')
print(sign_res.decode(encoding))


GLJBY6u95cLCSU3przaY5kq0RyTpA80Fdj8IRGxmZyFLUIWUV7H1lCBiPEb7it6yaKdmCR4pw4LtfQ0qGwssLIgjgj8bCAu4Y2twobx4ojldxWfV0NcIt/r04bPDTHFdFtneb1zSksJpXGZ28yeGYxfxYafiT8KjRJajNnQzAT4=


In [23]:


def pay_apply():
    url = test_url + pay_apply_api

    data = {
        "mch_id": mch_id,
        "user_id": "CCTEST20210906001",
        "user_ip": "192.168.1.1",
        "txn_seqno": 'CSP_A'+strftime('%Y%m%d%H%M%S', localtime()),
        "txn_time": strftime('%Y%m%d%H%M%S', localtime()),
        "order_amount": '1000',
        "order_info": 'xxdda py aa',
        "notify_url": "http://localtime.com",
        "pay_expire": "60",
        "pay_type":'STAGE_CODE_PAY',
        "stage_info":{
            "stage_num":'6'
        }
    }
    str_data= json.dumps(data)
    headers = {
        "timestamp": strftime('%Y%m%d%H%M%S', localtime()),
        "Signature-Data": rsa_sign(str_data).decode("utf-8"),
        "Signature-Type": "RSA",
        "mch_id": mch_id,
        "Content-Type": "application/json;charset=UTF-8"
    }

    r = requests.post(url=url, data=str_data, headers=headers)
    status = r.status_code
    print(r.json())


pay_apply()


{'ret_code': '0000', 'ret_msg': '交易成功', 'mch_id': '302112250000006404', 'txn_time': '20221123110549', 'txn_seqno': 'CSP_A20221123110549', 'platform_txno': '2022112346801882', 'order_amount': '1000.00', 'payload': '{"gateway_url":"http://101.231.114.212:10002/aps/dynamic/loading.html#loading?id=fa7f5b6434e1565f833d11d5b461e78656257dbbe46631a7e44a498177bfd46a"}'}
